<center><b>Effective Hamiltonians using PySCF</b></center>
<center> daan.verraes@ugent.be </center>

<div class="alert alert-block alert-info">
<b>Code benchmarking:</b> SrVO3
</div>

## Prelude

**PyWannier90:** (6.4.0 GCCore 10.3.0 IPython 7.25.0) \
module load pyWannier90/2021-12-07-foss-2021a \
module load matplotlib/3.4.2-foss-2021a

**ASE:** (6.4.0 GCCore 11.3.0 IPython 8.5.0) \
module load PySCF/2.1.1-foss-2022a \
module load ASE/3.22.1-foss-2022a

In [2]:
import numpy as np
from pyscf.pbc import gto, scf, cc, df, dft
from pyscf.lib import chkfile

import matplotlib.pyplot as plt
%matplotlib inline

ASE = True

if ASE:
    import pyscf.pbc.tools.pyscf_ase as pyscf_ase
    from ase.lattice import bulk
    from ase.dft.kpoints import sc_special_points as special_points, get_bandpath
else:
    import pywannier90

## Define unit cell 

In [3]:
# Lattice constant in [Å]:
a = 3.84652

cell = gto.Cell()
cell.atom = [['Sr',[0.0,0.0,0.0]],
             ['V',[a/2,a/2,a/2]],
             ['O',[a/2,a/2,0.0]],
             ['O',[a/2,0.0,a/2]],
             ['O',[0.0,a/2,a/2]]]
cell.a = np.eye(3) * a
cell.basis = {
    'Sr': gto.basis.parse("""
    Sr TZVP-MOLOPT-PBE-GTH-q10 TZVP-MOLOPT-GGA-GTH-q10
     1
     2 0 2 6 4 2 1
        14.48916111145965     2.25884524062801E-02    -1.34868194665989E-02     5.54347466759003E-03    -6.34061204601782E-02     2.06499646232493E-02     1.17383676982674E-02     1.14360210122985E-03
         2.15623936661967    -4.64342518925513E-01     3.59475029457052E-01    -4.70143743354400E-02     7.78871715629289E-01    -2.14974750089492E-01    -1.27979865553719E-01    -1.80202260985966E-02
         1.03613539116593     3.33230880551197E-01    -3.29393450960058E-01    -5.67977461098407E-02     1.17525562186450E-01     4.84134139687010E-01     3.20554567557613E-01     2.74056011019933E-01
         0.40204859937069     2.34326701416452E-01    -6.09501320740582E-01    -3.85922227366423E-01     5.04476105122795E-01     5.67539566699685E-01     4.94705577196014E-01     6.36550936632788E-01
         0.14809754979486     4.55723332187727E-01     4.50988125913452E-02     8.08224071603016E-01     3.38488174369872E-01     3.43157768722482E-01    -1.43576838157202E-01     6.56932449549031E-01
         0.04236584632575     6.40499967748533E-01     6.23353832490415E-01    -4.38600979293572E-01     8.03484404672735E-02     5.28313502165408E-01    -7.84461737365516E-01     2.96327400438876E-01
      #
    """),
    
    'V': gto.basis.parse("""
    V TZVP-MOLOPT-PBE-GTH-q13 TZVP-MOLOPT-GGA-GTH-q13
     1
     2 0 3 6 4 3 3 1
         7.33442219698472     8.27825761961714E-02     7.55011658547867E-02     2.24511235524045E-02     1.22895771891037E-01    -1.83579503640807E-01     3.57180401864135E-02     4.48474756328750E-02     1.40590997734590E-01    -1.07169041006575E-02     8.92275620154863E-02     6.55903605941630E-03
         3.71180865126492     3.47305780415066E-01     2.27060968905872E-01    -4.69617631181605E-02     1.08151153599133E-01     2.27317946465339E-01    -5.54382379165022E-02    -5.07409971207343E-02     3.54293109697276E-01    -1.45996628563614E-02     2.38719792100295E-01    -2.44760939719959E-02
         1.52733194631141    -5.48399617008117E-01    -3.65860899789597E-01     3.63857440971976E-01    -5.11789811341402E-01     6.92745649304493E-01    -3.32978321369422E-01     3.08502840955541E-01     5.71040303624255E-01    -3.02941302182320E-02     5.08321915100566E-01    -9.15243347888105E-02
         0.59629749846074    -1.01757099618302E-01    -4.72373457722919E-01     3.94696315466647E-01     7.74796088175320E-01     5.39593940777681E-01    -1.80643198550142E-01    -3.16142108387358E-01     6.66097442307729E-01    -1.24844025389668E-01    -2.62758620793206E-01    -6.80441969106978E-01
         0.21465533405075    -5.39057602096820E-01    -1.11723359065070E-01    -7.79365563349408E-01     2.43487845403239E-01     2.96134776157562E-01     6.48082113478254E-01    -6.04927743685262E-01     2.70864233148312E-01     6.81537901024263E-01    -5.75079047942911E-01    -7.26070149834146E-01
         0.06343681403080    -5.20428664019244E-01     7.57145890722093E-01     3.18908199530063E-01     2.27323199701933E-01     2.36370801609666E-01     6.57368899391423E-01     6.59057390890328E-01    -1.07565328827752E-01     7.20190476386105E-01     5.26214908075383E-01    -2.83206675230326E-02
         #
    """),
    
    'O': gto.basis.parse("""
    O TZVP-MOLOPT-PBE-GTH-q6 TZVP-MOLOPT-GGA-GTH-q6
     1
     2 0 2 5 3 3 1
        10.27704242406505    -1.49868256024880E-01     4.87596886258794E-02    -5.53669874919871E-01    -9.24779725877624E-02     6.96521980106397E-02     1.20727677711086E-01    -6.77536972887194E-02
         3.56405253880618    -1.50283608564945E-01     3.88821197655349E-02    -1.92378825213863E-01    -2.90527105694297E-01     2.15775427853434E-01     4.18617946994965E-01     1.82303508690924E-01
         1.31363563226775     5.39068797289606E-01    -1.25699044360202E-01     5.91746387586808E-01    -5.18398147191831E-01     3.67414070470753E-01     5.18264995222031E-01     3.34682703085916E-01
         0.48890324671371     8.02346121882746E-01    -7.57512538516891E-02    -5.49213059041492E-01    -6.26250099252499E-01     2.67796450584559E-01    -7.27741038771548E-01     9.10370319696758E-01
         0.15533153369410     1.43526566604286E-01     9.87204219659982E-01     6.81269826999462E-02    -4.96100795824058E-01    -8.61325430903662E-01     1.09445043302219E-01    -1.46247176017229E-01
      #
    """)}

cell.pseudo = {
    'Sr': gto.pseudo.parse("""
    Sr GTH-PBE-q10 GTH-PBE
    4    6
     0.48000000    2     6.81095035    -1.19610979
    3
     0.27588581    2    21.28971943    -7.89903416
                                       10.19760926
     0.28174086    2    11.70905135   -10.96577673
                                       12.97488200
     0.52108911    1     0.36053906
     #
    """),
    'V': gto.pseudo.parse("""
    V GTH-PBE-q13 GTH-PBE
    4    6    3
     0.37500000    2     7.47470354    -0.37026363
    3
     0.32779544    2     1.94087762     4.72568824
                                       -6.10083728
     0.24476590    2    -5.97816705     9.35863915
                                      -11.07329118
     0.24173879    1    -9.49989110
     #
    """),
    'O': gto.pseudo.parse("""
    O GTH-PBE-q6 GTH-PBE
    2    4
     0.24455430    2   -16.66721480     2.48731132
    2
     0.22095592    1    18.33745811
     0.21133247    0
     #
    """)}

cell.exp_to_discard = 0.1
cell.spin = 1
cell.build();

## PBE calculation

In [4]:
kmesh = [5,5,5]
kpts = cell.make_kpts(kmesh)
nkpts = kpts.shape[0]

run_scf = False

if run_scf:
    PBE = scf.KKS(cell, kpts)
    PBE = scf.addons.smearing_(PBE, sigma=.1, method='fermi')
    PBE.xc = 'PBE'
    PBE.verbose = 4
    PBE.init_guess_breaksym=False
    PBE.init_guess = "1e"
    PBE.chkfile = 'SrVO3_PBE.chk'
    PBE.run();

## Band structure

In [5]:
## Calculate PBE bands non-selfconsistently
run_bands = False

# Define high-symmetry k-paths:
hspoints = {
  "G": [0.0, 0.0, 0.0],
  "X": [0.0, 0.5, 0.0],
  "M": [0.5, 0.5, 0.0],
  "R": [0.5, 0.5, 0.5]}

G = hspoints['G']
X = hspoints['X']
M = hspoints['M']
R = hspoints['R']
kpath = [G, X, M, G, R, X]

# nscf along high-symmetry path:
if run_bands:
    
    band_kpts, kpath, sp_points = get_bandpath(kpath, cell.a, npoints=50);
    band_kpts = cell.get_abs_kpts(band_kpts);
    
    nscf = scf.KKS(cell, kpts)
    nscf.__dict__.update(chkfile.load("SrVO3_PBE.chk", 'scf'))
    E_nk = nscf.get_bands(np.array(band_kpts))
    E_F = nscf.get_fermi()
    
# Plot band structure (in eV):
plot_bands = False

if plot_bands:
    au2ev = 27.211386245988
    e_nk = (E_nk[0][0]- E_F[0])*au2ev

    plt.figure()
    nbands = cell.nao_nr()
    for n in range(nbands):
        plt.plot(kpath, [e[n] for e in e_nk], color='#4169E1')
    plt.plot([0, sp_points[-1]], [0, 0], '--', color='g')
    plt.xticks(sp_points, ['$%s$' % n for n in [r'\Gamma', 'X', 'M', r'\Gamma', 'R', 'X']])
    plt.xlabel('k-vector')
    plt.ylabel(r'$E-E_{F}$ [eV]')
    plt.ylim([-6,6])
    plt.grid()
    plt.savefig('Bands')
    plt.title('Band structure SrVO3')
    plt.show()

## Construct MLWFs

In [6]:
Run_90 = False # use *_MLWF.py script

if Run_90:    
    PBE = scf.KKS(cell, kpts)
    PBE.__dict__.update(chkfile.load("SrVO3_PBE.chk", 'scf'))

    pywannier90.save_kmf(PBE, 'chk_W90')
    kmf = pywannier90.load_kmf('chk_W90')

    # Construct MLWFs:
    num_wann = 3
    keywords = \
    """
    # PBE energy window
    exclude_bands : 1-20,24-101

    begin projections
     V:dxy;dxz;dyz
    end projections

    write_u_matrices = .TRUE.
    """
    w90 = pywannier90.W90(kmf, cell, kmesh, num_wann, other_keywords=keywords)
    w90.kernel()    

    # Export the MWLFs in the .xsf format for plotting with VESTA:
    w90.plot_wf(supercell=kmesh, grid=[20,20,20])

    # Export wannier90.mmn, wannier90.amn, wannier90.eig matrix and then run a wannier90 using these:
    w90.export_AME()
    w90.kernel(external_AME='wannier90')

    w90.WANPROJ()

    #TODO: Plot bands using Wannier interpolation and compare to bands from SCF

## Hopping parameters

In [7]:
N_k = 5**3
N_b = 3
N_w = 3

au2angst = 0.529177
nn = np.array([[0,0,0], [1,0,0], [1,1,0], [2, 0, 0]])*3.84652

loaded_arr = np.loadtxt("WANPROJ_opt.txt").view(complex)
WANPROJ_opt = loaded_arr.reshape(loaded_arr.shape[0], loaded_arr.shape[1] // N_b, N_b)

loaded_arr = np.loadtxt("WANPROJ.txt").view(complex)
WANPROJ = loaded_arr.reshape(loaded_arr.shape[0], loaded_arr.shape[1] // N_b, N_b)

T = WANPROJ
        
Eig = open('wannier90.eig', "r")
Eig = Eig.read()
Eig = np.array([x for x in Eig.split(" ") if x != '' and x != '\n'])

E = np.zeros((N_k, N_b),dtype=complex)
for k in range(N_k):
    for b in range(N_b):
            E[k, b] = float(Eig[2+3*b+3*N_b*k])

t = np.zeros((len(nn), N_w, N_w))
for R in range(len(nn)):    
    for i in range(N_w):
        for j in range(N_w):
            for n in range(N_b):
                for k in range(N_k):
                    t[R, i, j] -= ((np.conj(T[k, i, n])*(E[k][n])*T[k, j, n])*np.exp(1j*np.dot(kpts[k]/au2angst, nn[R]))).real
    print('Hopping matrix [eV] to', nn[R],':\n \n', (1/N_k)*t[R], '\n')

Hopping matrix [eV] to [0. 0. 0.] :
 
 [[-1.47616167e+01 -2.86193185e-11 -2.46584300e-11]
 [-2.86192900e-11 -1.47616158e+01 -2.50309995e-12]
 [-2.46583873e-11 -2.50313548e-12 -1.47616155e+01]] 

Hopping matrix [eV] to [3.84652 0.      0.     ] :
 
 [[ 2.81303453e-01  3.13913365e-08  3.63411612e-11]
 [-2.86788158e-08  3.00791336e-02 -3.58241423e-08]
 [ 3.63157018e-11  3.85134640e-08  2.81303657e-01]] 

Hopping matrix [eV] to [3.84652 3.84652 0.     ] :
 
 [[ 1.72991772e-03 -3.89446406e-03 -1.50805137e-08]
 [-3.89447039e-03  1.72992655e-03  2.01157832e-08]
 [ 1.45766155e-08 -1.95121683e-08  5.94012611e-02]] 

Hopping matrix [eV] to [7.69304 0.      0.     ] :
 
 [[-9.95284361e-03 -3.00737165e-09  8.73814088e-12]
 [ 2.91145805e-09  3.61746196e-04  3.58069514e-09]
 [ 8.73325945e-12 -3.67569401e-09 -9.95283320e-03]] 



## Hubbard parameters from cRPA

In [ ]:
# TODO